# Test Notebook: barwerte.py

Interaktive Tests der versicherungsmathematischen Funktionen aus dem migrierten Excel-Tarifrechner.

## Inhaltsverzeichnis
1. Import und Setup
2. Grundfunktionen (Diskont, Abzugsglied)
3. Sterbetafel laden und testen
4. Ueberlebenswahrscheinlichkeiten
5. Leibrenten-Barwerte
6. Versicherungsleistungen
7. Beispielrechnung: Kapitallebensversicherung

## 1. Import und Setup

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

# Import der barwerte-Funktionen
from barwerte import (
    Sterbetafel, diskont, npx, nqx, abzugsglied,
    ax, ax_k, axn, axn_k, nax_k,
    nAx, nEx, ag_k,
    MAX_ALTER
)

print(f"Maximales Alter: {MAX_ALTER}")
print("Module erfolgreich geladen!")

## 2. Grundfunktionen

In [ ]:
# Test Diskontierungsfaktor
zins = 0.0175  # 1,75%
v = diskont(zins)

print(f"Diskontierungsfaktor v bei i={zins:.4%}: {v:.8f}")
print(f"Kontrolle: 1/(1+i) = {1/(1+zins):.8f}")

In [ ]:
# Test Abzugsglied (Woolhouse-Naeherung)
print("Abzugsglied beta(k,i) fuer verschiedene Zahlungsweisen:")
print(f"Zinssatz: {zins:.4%}\n")

zahlungsweisen = {
    1: 'jaehrlich',
    2: 'halbjaehrlich', 
    4: 'vierteljaehrlich',
    12: 'monatlich'
}

for zw, beschreibung in zahlungsweisen.items():
    beta = abzugsglied(zw, zins)
    print(f"k={zw:2d} ({beschreibung:20s}): beta = {beta:.6f}")

## 3. Sterbetafel laden

In [ ]:
# Pfad zur Sterbetafel (bitte anpassen!)
tafel_pfad = Path("C:/Users/chares/Documents/Notebooks/xlsmToPython/data/Tafeln.xlsx")

try:
    st = Sterbetafel(tafel_pfad)
    print(f"Sterbetafel erfolgreich geladen!")
    print(f"\nVerfuegbare Tafeln: {st.verfuegbare_tafeln}")
except FileNotFoundError:
    print(f"FEHLER: Datei nicht gefunden: {tafel_pfad}")
    print("Bitte Pfad anpassen!")
    st = None

In [ ]:
# Test: Einzelne qx-Werte abrufen
if st is not None:
    alter = 40
    sex = 'M'
    tafel = 'DAV1994_T'
    
    qx_wert = st.qx(alter, sex, tafel)
    px_wert = 1 - qx_wert
    
    print(f"Alter: {alter}, Geschlecht: {sex}, Tafel: {tafel}")
    print(f"\nqx({alter}) = {qx_wert:.8f} ({qx_wert*100:.6f}%)")
    print(f"px({alter}) = {px_wert:.8f} ({px_wert*100:.6f}%)")

In [ ]:
# Test: qx-Vektor fuer mehrere Jahre
if st is not None:
    alter = 40
    n = 10
    sex = 'M'
    tafel = 'DAV1994_T'
    
    qx_vec = st.qx_vec(alter, n, sex, tafel)
    
    print(f"qx-Vektor fuer {n} Jahre ab Alter {alter}:\n")
    
    df = pd.DataFrame({
        'Alter': range(alter, alter + n),
        'qx': qx_vec,
        'px': 1 - qx_vec
    })
    
    print(df.to_string(index=False))

## 4. Ueberlebenswahrscheinlichkeiten

In [ ]:
# Test: n-jahrige Ueberlebenswahrscheinlichkeit
if st is not None:
    alter = 40
    sex = 'M'
    tafel = 'DAV1994_T'
    
    print(f"Parameter: Alter={alter}, Sex={sex}, Tafel={tafel}\n")
    print("n-jahrige Ueberlebenswahrscheinlichkeit npx:\n")
    
    laufzeiten = [1, 5, 10, 15, 20, 25, 30]
    
    ergebnisse = []
    for n in laufzeiten:
        n_px = npx(alter, n, sex, tafel, st)
        n_qx = nqx(alter, n, sex, tafel, st)
        ergebnisse.append({
            'n': n,
            'npx': n_px,
            'nqx': n_qx
        })
    
    df = pd.DataFrame(ergebnisse)
    print(df.to_string(index=False))

## 5. Leibrenten-Barwerte

In [ ]:
# Test: Lebenslange Leibrente
if st is not None:
    alter = 40
    sex = 'M'
    tafel = 'DAV1994_T'
    zins = 0.0175
    
    print(f"Parameter: Alter={alter}, Sex={sex}, Tafel={tafel}, Zins={zins:.4%}\n")
    print("Lebenslange vorschuessige Leibrente:\n")
    
    # Jaehrliche Zahlung
    ax_1 = ax(alter, sex, tafel, zins, st)
    print(f"ax({alter}) [jaehrlich] = {ax_1:.6f}")
    
    # Mit verschiedenen Zahlungsweisen
    print("\nMit unterjahrigen Zahlungen:")
    for zw in [1, 2, 4, 12]:
        ax_k_wert = ax_k(alter, sex, tafel, zins, zw, st)
        print(f"ax^({zw:2d})({alter}) = {ax_k_wert:.6f}")

In [ ]:
# Test: Temporaere Leibrente
if st is not None:
    alter = 40
    n = 20
    sex = 'M'
    tafel = 'DAV1994_T'
    zins = 0.0175
    
    print(f"Temporaere Leibrente (n={n} Jahre):\n")
    
    # Jaehrliche Zahlung
    axn_1 = axn(alter, n, sex, tafel, zins, st)
    print(f"ax:{n}({alter}) [jaehrlich] = {axn_1:.6f}")
    
    # Mit verschiedenen Zahlungsweisen
    print("\nMit unterjahrigen Zahlungen:")
    for zw in [1, 2, 4, 12]:
        axn_k_wert = axn_k(alter, n, sex, tafel, zins, zw, st)
        print(f"ax:{n}^({zw:2d})({alter}) = {axn_k_wert:.6f}")

## 6. Versicherungsleistungen

In [ ]:
# Test: Todesfallversicherung und Erlebensfallversicherung
if st is not None:
    alter = 40
    n = 20
    sex = 'M'
    tafel = 'DAV1994_T'
    zins = 0.0175
    
    print(f"Parameter: Alter={alter}, Laufzeit={n} Jahre\n")
    
    # Todesfallversicherung
    Ax_n = nAx(alter, n, sex, tafel, zins, st)
    print(f"Todesfallversicherung Ax:{n}({alter}) = {Ax_n:.8f}")
    
    # Erlebensfallversicherung
    Ex_n = nEx(alter, n, sex, tafel, zins, st)
    print(f"Erlebensfallversicherung Ex:{n}({alter}) = {Ex_n:.8f}")
    
    # Summe (gemischte Versicherung)
    summe = Ax_n + Ex_n
    print(f"\nSumme (Ax + Ex) = {summe:.8f}")

## 7. Beispielrechnung: Kapitallebensversicherung

Nachbildung der Berechnung aus dem Excel-Tarifrechner.

In [ ]:
if st is not None:
    # Vertragsdaten (wie im Excel-Screenshot)
    x = 40           # Alter
    sex = 'M'        # Geschlecht
    n = 30           # Laufzeit
    vs = 100000.00   # Versicherungssumme in EUR
    zins = 0.0175    # Zinssatz (1,75%)
    tafel = 'DAV1994_T'
    zw = 12          # monatliche Zahlung
    
    print("="*70)
    print(" "*15 + "KAPITALLEBENSVERSICHERUNG")
    print("="*70)
    
    print("\nVertragsdaten:")
    print(f"  Alter:                {x}")
    print(f"  Geschlecht:           {sex}")
    print(f"  Laufzeit:             {n} Jahre")
    print(f"  Versicherungssumme:   {vs:,.2f} EUR")
    print(f"  Zinssatz:             {zins:.4%}")
    print(f"  Tafel:                {tafel}")
    print(f"  Zahlungsweise:        {zw}x pro Jahr (monatlich)")
    
    # Berechnung Barwerte (pro 1 EUR Versicherungssumme)
    print("\n" + "-"*70)
    print("Barwerte (pro 1 EUR Versicherungssumme):")
    print("-"*70)
    
    # Todesfallleistung
    Bxt = nAx(x, n, sex, tafel, zins, st)
    print(f"\nTodesfall-Barwert (Bxt):")
    print(f"  Ax:{n}({x}) = {Bxt:.8f}")
    
    # Erlebensfallleistung
    Bxe = nEx(x, n, sex, tafel, zins, st)
    print(f"\nErlebensfall-Barwert (Bxe):")
    print(f"  Ex:{n}({x}) = {Bxe:.8f}")
    
    # Beitragsbarwert
    Pxt = axn_k(x, n, sex, tafel, zins, zw, st)
    print(f"\nBeitrags-Barwert (Pxt):")
    print(f"  ax:{n}^({zw})({x}) = {Pxt:.8f}")
    
    # Gesamtleistung
    BzB = Bxt + Bxe
    print(f"\nGesamtleistung pro 1 EUR VS (BzB = Bxt + Bxe):")
    print(f"  {BzB:.8f}")
    
    # Nettoprämie nach Äquivalenzprinzip
    print("\n" + "-"*70)
    print("Netto-Praemie (Aequivalenzprinzip):")
    print("-"*70)
    
    # Pro 1 EUR Versicherungssumme
    praemie_pro_euro = BzB / Pxt
    print(f"\nPraemie pro 1 EUR VS:")
    print(f"  {praemie_pro_euro:.10f} EUR")
    
    # Bei gegebener Versicherungssumme
    jahrespram = praemie_pro_euro * vs
    print(f"\nJahrespramie bei VS = {vs:,.2f} EUR:")
    print(f"  {jahrespram:,.2f} EUR")
    
    # Monatsprämie
    monatspraem = jahrespram / 12
    print(f"\nMonatspraemie:")
    print(f"  {monatspraem:,.2f} EUR")
    
    print("\n" + "="*70 + "\n")

## Weitere Tests und Experimente

Hier koennen Sie weitere Tests und Berechnungen durchfuehren.

In [ ]:
# Ihr Code hier...